# Plotting Example Notebook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# define some column groups we already know
id_cols = ["locationId", "longitude", "latitude"]
baseline_cols = [f"FR_fireProbability_1995_baseline_{v}" for v in ["lower", "mean", "upper"]]
ssp585_mean_cols = [f"FR_fireProbability_{y}_ssp585_mean" for y in range(2020, 2045, 5)]
usecols = id_cols + baseline_cols + ssp585_mean_cols

In [ ]:
# read input csv (only grab a subset of columns and a few rows)
df = pd.read_csv(
    "s3://jupiter-climatescoreglobal-eos/reports/MMC/MMC_results_raw_fire_pt4.csv",
    usecols=usecols,
    nrows=7,
)
df

In [ ]:
# some UQ checks on Baseline
plt.figure(figsize=(14, 8))
for line, loc_id in zip(df[baseline_cols].values, df.locationId.values):
    if all(np.isnan(line)):
        print(f"All nan: ID: {loc_id}")
    elif all(line < 0.0001):
        print(f"All 0s: ID: {loc_id}")
    else:
        condition_max = np.max(line) < 0.6
        condition_mean_low_diff = 0.0 <= line[1] - line[0] < 0.25
        condition_high_mean_diff = 0.0 <= line[2] - line[1] < 0.25
        if all([condition_max, condition_mean_low_diff, condition_high_mean_diff]):
            plt.plot(line, '-o', label=loc_id)
        else:
            print(f"No conditions met: ID: {loc_id}")
plt.legend()
plt.title("Baseline UQ Checks")
plt.show()

In [ ]:
# some year checks on ssp585 mean
plt.figure(figsize=(14, 8))
for line, loc_id in zip(df[ssp585_mean_cols].values, df.locationId.values):
    if all(np.isnan(line)):
        print(f"All nan: ID: {loc_id}")
    elif all(line < 0.0001):
        print(f"All 0s: ID: {loc_id}")
    else:
        diff = np.diff(line)
        condition_neg_diff = not any(diff < 0)
        condition_pos_diff_big = not any(diff > 0.25)
        if all([condition_neg_diff, condition_pos_diff_big]):
            plt.plot(line, '-o', label=loc_id)
        else:
            print(f"No conditions met: ID: {loc_id}")
plt.legend()
plt.title("ssp585_mean Year Checks")
plt.show()

In [ ]:
# plot a single location by id
loc = 607
plt.plot(df.query("locationId == @loc")[ssp585_mean_cols].transpose().values, "-o")
plt.title(f"ssp585 mean, ID: {loc}")
plt.show()